In [1]:
%%capture

!apt-get update
!apt-get install -y openjdk-8-jdk-headless -qq
!apt-get install maven -qq

!curl -L "https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz" > spark-2.4.5-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark py4j
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql import functions
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
#from pyspark.sql.Column import isNull
from pyspark.sql.functions import lit
spark = SparkSession.builder.master("local[*]").config("spark.driver.memory", "16g").getOrCreate()

In [2]:

from pyspark import SparkConf, SparkContext
conf = SparkConf().setAppName("Read File")
sc = SparkContext.getOrCreate(conf=conf)   # we provide configuration


In [ ]:
#1 No of students in the file

rdd = sc.textFile('StudentData.csv')
#rdd.collect()
headers=rdd.first() # to get header row
rdd=rdd.filter(lambda x: x!= headers)
rdd.count()   # now header will be removed
rdd.collect()


In [17]:

#2 total marks by male and female students

rdd = sc.textFile('StudentData.csv')
headers=rdd.first() # to get header row
rdd=rdd.filter(lambda x: x!= headers)
rdd=rdd.map(lambda x:x.split(','))
rdd2=rdd
rdd2=rdd2.map(lambda x: (x[1],int(x[5])))
rdd2.reduceByKey(lambda x,y: x+y).collect()


[('Female', 29636), ('Male', 30461)]

In [18]:
#3 pass and fail

rdd3=rdd
passed=rdd3.filter(lambda x:int(x[5])>50).count()
failed=rdd3.filter(lambda x:int(x[5])<=50).count()
print(passed,failed)

630 370


In [19]:
#4 total enrollment per course

rdd4=rdd
rdd4=rdd4.map(lambda x:(x[3],1))
rdd4.reduceByKey(lambda x,y:x+y).collect()

[('DB', 157),
 ('Cloud', 192),
 ('PF', 166),
 ('MVC', 157),
 ('OOP', 152),
 ('DSA', 176)]

In [21]:
#5 Total marks per course

rdd5=rdd
rdd5=rdd5.map(lambda x: (x[3], int(x[5])))
rdd5.reduceByKey(lambda x,y:x+y).collect()



[('DB', 9270),
 ('Cloud', 11443),
 ('PF', 9933),
 ('MVC', 9585),
 ('OOP', 8916),
 ('DSA', 10950)]

In [32]:
#6 Average
rdd6=rdd
rdd6=rdd6.map(lambda x: (x[3], (int(x[5]),1)))
rdd6=rdd6.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))
#rdd6= rdd6.map(lambda x:(x[0],round(x[1][0]/x[1][1])))
#rdd6.collect()

# alternate method
# i.e. remove this step rdd6= rdd6.map(lambda x:(x[0],round(x[1][0]/x[1][1])))

rdd6.mapValues(lambda x:x[0]/x[1]).collect()
# we can use mapValues with reduceByKey also.



[('DB', 59.044585987261144),
 ('Cloud', 59.598958333333336),
 ('PF', 59.83734939759036),
 ('MVC', 61.05095541401274),
 ('OOP', 58.6578947368421),
 ('DSA', 62.21590909090909)]

In [34]:
#7 Maximum and minimum marks of course
rdd7=rdd
rdd7=rdd7.map(lambda x:(x[3],int(x[5])))
print(rdd7.reduceByKey(lambda x,y: x if x>y else y).collect())  # maximum marks
print(rdd7.reduceByKey(lambda x,y: x if x<y else y).collect())  # minimummm marks


[('DB', 98), ('Cloud', 99), ('PF', 99), ('MVC', 99), ('OOP', 99), ('DSA', 99)]
[('DB', 20), ('Cloud', 20), ('PF', 20), ('MVC', 22), ('OOP', 20), ('DSA', 20)]


In [38]:
#8 Average age of male and female student

rdd8=rdd
rdd8=rdd8.map(lambda x:( x[1], (int(x[0]),1)))
rdd8=rdd8.reduceByKey(lambda x,y: (x[0]+y[0] , x[1]+y[1]) )

rdd8.mapValues(lambda x: round(x[0]/x[1])).collect()

[('Female', 28), ('Male', 29)]